In [2]:
%cd C:/Users/gioc4/Documents/python-benchmark

C:\Users\gioc4\Documents\python-benchmark


In [15]:
import pandas as pd
from provider_scorecard.benchmark import Benchmark

# load data
df = pd.read_csv("C:/Users/gioc4/Documents/blog/data/sparcs2.zip")
 
# transform
# get only the top 30 drg codes
df = df[df['Hospital County'] == 'Suffolk']

# some recoding
df['Length of Stay'] = [120 if x == '120 +' else x for x in df['Length of Stay']]
df['Length of Stay'] = df['Length of Stay'].astype(int)
df['APR DRG Code'] = df['APR DRG Code'].astype(str)

# top 30 drg codes
drg = df.groupby('APR DRG Code')['APR DRG Code'].count().sort_values(ascending=False).head(30)
drg = drg.astype(str)

# final dataframe
df = df[df['APR DRG Code'].isin(drg.index)]

C:\Users\gioc4\AppData\Local\Temp\ipykernel_16100\3931642226.py:6: DtypeWarning: Columns (10,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("C:/Users/gioc4/Documents/blog/data/sparcs2.zip")


In [16]:
df['Facility Name'].unique()

array(['Long Island Community Hospital', 'Peconic Bay Medical Center',
       'South Shore University Hospital', 'Huntington Hospital',
       'Stony Brook University Hospital',
       'Stony Brook Southampton Hospital',
       'Stony Brook Eastern Long Island Hospital',
       'Good Samaritan Hospital Medical Center', 'St. Charles Hospital',
       'St Catherine of Siena Hospital',
       'John T Mather Memorial Hospital of Port Jefferson New York Inc'],
      dtype=object)

In [18]:
df.groupby('Facility Name')['Facility Name'].count().sort_values(ascending=False)

Facility Name
Stony Brook University Hospital                                   17831
Good Samaritan Hospital Medical Center                            13667
South Shore University Hospital                                   12709
Huntington Hospital                                               10601
St. Charles Hospital                                               7790
Peconic Bay Medical Center                                         5468
St Catherine of Siena Hospital                                     5314
Long Island Community Hospital                                     4574
John T Mather Memorial Hospital of Port Jefferson New York Inc     4178
Stony Brook Southampton Hospital                                   3057
Stony Brook Eastern Long Island Hospital                            847
Name: Facility Name, dtype: int64

In [27]:
# input features
xfeat = [
    "Age Group",
    "Gender",
    "Race",
    "Length of Stay",
    "Type of Admission",
    "APR DRG Code",
    "APR Severity of Illness Description",
    "APR Risk of Mortality",
]

# evaluation features
efeat = ["Total Charges", "Total Costs", "Payment Typology 1"]

# focal hospitals
hosp = [
    'Stony Brook University Hospital',
    'Good Samaritan Hospital Medical Center',
    'South Shore University Hospital',
    'Huntington Hospital',
    'St. Charles Hospital',
    'Long Island Community Hospital'
]

In [28]:
res_list = []

# loop through hospitals, store dataframes in list
for h in hosp:
    tr = df["Facility Name"] == h

    bch = Benchmark(
        data=df, focal_indicator=tr, predictor_features=xfeat, evaluation_features=efeat
    )
    bch.fit(lrate=0.1, nest=250)
    bch.evaluate()

    res_list.append(
        pd.DataFrame(
            {"Hospital": h, "Outcome": bch.Xeval.columns, "Value": bch.outcomes}
        )
    )

In [29]:
# concat into single pandas dataframe
results = pd.concat(res_list, axis=0)

In [44]:
res_list[5]

,Hospital,Outcome,Value
0,Long Island Community Hospital,Total Charges,-9788.00
1,Long Island Community Hospital,Total Costs,-7248.00
2,Long Island Community Hospital,Payment Typology 1_Blue Cross/Blue Shield,-0.02
3,Long Island Community Hospital,Payment Typology 1_Department of Corrections,-0.00
4,Long Island Community Hospital,Payment Typology 1_Federal/State/Local/VA,-0.01
5,Long Island Community Hospital,"Payment Typology 1_Managed Care, Unspecified",-0.03
6,Long Island Community Hospital,Payment Typology 1_Medicaid,-0.11
7,Long Island Community Hospital,Payment Typology 1_Medicare,0.04
8,Long Island Community Hospital,Payment Typology 1_Miscellaneous/Other,-0.01
9,Long Island Community Hospital,Payment Typology 1_Private Health Insurance,0.15
